# 📘 DevOps 업무전환 - Epic 생성
- **목적**: DevOps 전환을 위해 작성한 항목들을 에픽과 스토리로 구분하고, 에픽의 우선순위를 산정
- **입력 데이터**: https://www.notion.so/devops-77d2cc12fdb74c298e98cc8a45d812db?source=copy_link
- **주요 기능**
  - 항목들 에픽 구성(합치거나, 스토리로 만들거나, 합쳐서 하나의 에픽으로 만들거나)
  - 에픽 우선 순위 산정
- **작성자**: 김동영
- **작성일**: 2025-08-01

# 0. 환경 설정

In [1]:
import os, sys
sys.path.append(os.path.abspath("../src"))

import dotenv
import json

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from epic.services import EpicGeneratorAgent
# from epic.prompts import TASK_TO_EPIC_CONVERTER_PROMPT

In [2]:
dotenv.load_dotenv('../.env')

True

# 1. 데이터 정제

In [3]:
# 에픽 생성 llm 모델 생성
epic_agent = EpicGeneratorAgent(
    openai_api_key = os.getenv("OPENAI_API_KEY")
)

/Users/visuworks/Documents/07_n8n/project_manage_agent/src/epic/services.py:19: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(


In [4]:
# 프롬프트 재수정
TASK_TO_EPIC_CONVERTER_PROMPT = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template(
        """
        [Role] 너는 숙련된 프로젝트 매니저이자 업무 분석 전문가다.
        [Goal] 정리되지 않은 업무 리스트를 분석하여 관련 업무들을 그룹핑하고, 각 그룹을 하나의 Epic으로 변환하는 것.
        [Instruction] 
          1. 주어진 업무 리스트를 분석하여 기능적으로 관련된 업무들을 식별한다.
          2. 유사하거나 같은 기능 영역의 업무들을 하나의 Epic으로 그룹핑한다.
          3. 각 Epic에 포함되지 않은 개별 업무는 별도의 Epic으로 생성한다.
          4. Epic 제목은 해당 기능 영역을 대표하는 명확한 이름으로 작성한다.
        [Constraints]
          - 출력은 반드시 [Output Format]을 따라야 한다.
          - Epic은 하나 이상의 스프린트에 걸쳐 진행되는 큰 기능 단위여야 한다.
          - 불필요한 설명, 주석, 마크다운 없이 JSON만 출력한다.
          - 최대 {max_epics}개의 Epic을 생성한다.
          - 각 업무들로 에픽을 구성할 때 최소한의 에픽으로 구성한다
          - acceptance_criteria는 그룹핑된 업무들을 기반으로 구체적이고 측정 가능한 기준으로 작성한다.
          - 개발자의 업무 영역만 작성한다.
          - 각 Epic의 included_tasks에는 해당 Epic에 포함된 원본 업무들을 나열한다.
        [Output Format]
        [
          {{
            "title": "에픽 제목",
            "description": "에픽 설명", 
            "business_value": "비즈니스 가치",
            "priority": "High|Medium|Low",
            "acceptance_criteria": ["수락 기준 1", "수락 기준 2", "수락 기준 3"],
            "included_tasks": ["포함된 원본 업무 1", "포함된 원본 업무 2"],
          }}
        ]
        """
    ),

    HumanMessagePromptTemplate.from_template(
        """
        [Input] 정리되지 않은 업무 리스트:
        {user_input}
        
        [Context] {project_info}
        """
    )
    ]
)

In [5]:
# llm prompt에 포함할 유저 지시내용 작성(프로세스의 업무사항)
# - 입력받은 업무를 에픽으로 변환, 여러개를 하나의 에픽으로 구성하고 구성항목을 하위항목(스토리)으로 두거나, 하나의 업무를 하나의 에픽으로 구성
user_input = """
- GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성
    - PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행
    - Dockerfile 실행 오류 및 종속성 검증 포함
- PR 병합은 `main`, `develop` 브랜치로만 제한 (브랜치 보호 규칙 설정)
- PR 템플릿 제공 및 코드 리뷰 프로세스 문서화
- 각 브랜치/환경별 `.env`, Secrets Manager, Parameter Store 등 사용
- CodeBuild 연동 (`buildspec.yml`)
- DockerHub 또는 ECR에 결과 이미지 자동 푸시
- 결과 Slack/이메일/웹훅 알림 연동
- 배포 트리거: GitHub Webhook, PR 병합, 수동 승인
- **배포 전략 선택**
    - 블루그린 배포 (Blue/Green)
    - 카나리아 배포
    - 롤링 배포
- AWS IAM Role, SSH 키를 통한 보안 배포
- 배포 전후 헬스체크 및 자동 롤백 구성
- 배포 이력/승인 기록 관리
- **정책 생성**
    - 파이프라인/워크플로 명명 규칙 (`ci-<service>`, `deploy-<env>`)
    - 신규 프로젝트 기본 템플릿 및 체크리스트 제공
    - 배포 승인 프로세스 및 롤백 기준 명시
"""

In [6]:
generated_epics_str = epic_agent._generate_epics_with_llm(
    prompt = TASK_TO_EPIC_CONVERTER_PROMPT,
    user_input = user_input,
    project_info= '',
    max_epics=10
)

INFO:epic.services:에픽 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:epic.services:에픽 생성 완료


In [ ]:
parsed = json.loads(generated_epics_str)

In [20]:
# llm prompt에 포함할 유저 지시내용 작성(프로세스의 업무사항)
# - 입력받은 업무를 에픽으로 변환, 여러개를 하나의 에픽으로 구성하고 구성항목을 하위항목(스토리)으로 두거나, 하나의 업무를 하나의 에픽으로 구성
# 하위 항목 구성 수정 
user_input_1 = """
- GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성(PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행, Dockerfile 실행 오류 및 종속성 검증 포함
- PR 병합은 `main`, `develop` 브랜치로만 제한 (브랜치 보호 규칙 설정)
- 각 브랜치/환경별 `.env`, Secrets Manager, Parameter Store 등 사용
- CodeBuild 연동 (`buildspec.yml`)
- DockerHub 또는 ECR에 결과 이미지 자동 푸시
- 결과 Slack/이메일/웹훅 알림 연동
- 배포 트리거 전략 세운 후 적용( GitHub Webhook, PR 병합, 수동 승인 등)
- **배포 전략 선택 (**블루그린 배포 (Blue/Green), 카나리아 배포, 롤링 배포 중에 선택 후 적용)
- AWS IAM Role, SSH 키를 통한 보안 배포
- 배포 전후 헬스체크 및 자동 롤백 구성
- 배포 이력/승인 기록 관리
- **정책 생성(** 파이프라인/워크플로 명명 규칙 , 신규 프로젝트 기본 템플릿 및 체크리스트 제공, 배포 승인 프로세스 및 롤백 기준 명시, PR 템플릿 제공 및 코드 리뷰 프로세스 문서화 등)
"""

In [21]:
generated_epics_str_1 = epic_agent._generate_epics_with_llm(
    prompt = TASK_TO_EPIC_CONVERTER_PROMPT,
    user_input = user_input_1,
    project_info= '',
    max_epics=10
)

INFO:epic.services:에픽 생성 시작


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:epic.services:에픽 생성 완료


In [22]:
parsed_1 = json.loads(generated_epics_str_1)
print(parsed_1)

[{'title': 'CI/CD 자동화 구성', 'description': 'GitHub Actions 또는 Jenkins를 활용하여 End-to-End 자동화 구성 및 배포 전략 수립', 'business_value': '개발 프로세스의 효율성을 높이고, 배포 오류를 줄여 신속한 피드백을 제공', 'priority': 'High', 'acceptance_criteria': ['End-to-End 자동화가 성공적으로 구성되어 PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동 실행됨', 'Dockerfile 실행 오류 및 종속성 검증이 성공적으로 수행됨', '결과 이미지가 DockerHub 또는 ECR에 자동으로 푸시됨'], 'included_tasks': ['GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성(PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행, Dockerfile 실행 오류 및 종속성 검증 포함)', 'DockerHub 또는 ECR에 결과 이미지 자동 푸시']}, {'title': '브랜치 보호 및 관리', 'description': 'PR 병합을 위한 브랜치 보호 규칙 설정 및 환경별 비밀 관리', 'business_value': '코드 품질을 유지하고, 배포 안정성을 높임', 'priority': 'Medium', 'acceptance_criteria': ['PR 병합이 `main`, `develop` 브랜치로만 제한됨', '각 브랜치/환경별 `.env`, Secrets Manager, Parameter Store 등이 성공적으로 설정됨'], 'included_tasks': ['PR 병합은 `main`, `develop` 브랜치로만 제한 (브랜치 보호 규칙 설정)', '각 브랜치/환경별 `.env`, Secrets Manager, Parameter Store 등 사용']}, {'title': '배포 전략 및

In [23]:
for x in parsed:
    print(x['title'])

CI/CD 자동화 구성
브랜치 보호 및 PR 관리
배포 전략 및 트리거 설정
보안 배포 및 헬스체크
배포 이력 및 승인 관리
환경 설정 및 비밀 관리
알림 시스템 구축
정책 및 템플릿 관리


In [25]:
for x in parsed_1:
    print(x['title'])

CI/CD 자동화 구성
브랜치 보호 및 관리
배포 전략 및 헬스체크
배포 승인 및 이력 관리
배포 정책 및 템플릿 생성


In [26]:
# 하위 항목 구성 수정 
# *및 띄어쓰기 삭제
user_input_2 = """GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성(PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행, Dockerfile 실행 오류 및 종속성 검증 포함), PR 병합은 main, develop 브랜치로만 제한 (브랜치 보호 규칙 설정), 각 브랜치/환경별 .env, Secrets Manager, Parameter Store 등 사용, CodeBuild 연동 (buildspec.yml), DockerHub 또는 ECR에 결과 이미지 자동 푸시, CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동, 배포 트리거 전략 세운 후 적용(GitHub Webhook, PR 병합, 수동 승인 등), 배포 전략 선택(블루그린 배포(Blue/Green), 카나리아 배포, 롤링 배포 중에 선택 후 적용), AWS IAM Role, SSH 키를 통한 보안 배포, 배포 전후 헬스체크 및 자동 롤백 구성, 배포 이력/승인 기록 관리, 정책 생성(파이프라인/워크플로 명명 규칙, 신규 프로젝트 기본 템플릿 및 체크리스트 제공, 배포 승인 프로세스 및 롤백 기준 명시, PR 템플릿 제공 및 코드 리뷰 프로세스 문서화 등)"""

In [27]:
generated_epics_str_2 = epic_agent._generate_epics_with_llm(
    prompt = TASK_TO_EPIC_CONVERTER_PROMPT,
    user_input = user_input_2,
    project_info= '',
    max_epics=10
)

INFO:epic.services:에픽 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:epic.services:에픽 생성 완료


In [28]:
parsed_2 = json.loads(generated_epics_str_1)
print(parsed_2)

[{'title': 'CI/CD 자동화 구성', 'description': 'GitHub Actions 또는 Jenkins를 활용하여 End-to-End CI/CD 자동화 프로세스를 구성한다.', 'business_value': '개발 및 배포 프로세스의 효율성을 높이고, 코드 품질을 유지하여 빠른 피드백을 제공한다.', 'priority': 'High', 'acceptance_criteria': ['PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동으로 실행된다.', 'Dockerfile 실행 오류 및 종속성 검증이 포함된다.', 'CI/CD 결과가 Slack/이메일/웹훅으로 알림된다.'], 'included_tasks': ['GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성', 'PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행', 'Dockerfile 실행 오류 및 종속성 검증 포함', 'CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동']}, {'title': '브랜치 보호 및 관리', 'description': '코드 품질을 보장하기 위해 특정 브랜치에 대한 보호 규칙을 설정하고 관리한다.', 'business_value': '코드의 안정성을 높이고, 팀원 간의 협업을 원활하게 한다.', 'priority': 'Medium', 'acceptance_criteria': ['PR 병합은 main, develop 브랜치로만 제한된다.', '브랜치 보호 규칙이 설정된다.'], 'included_tasks': ['PR 병합은 main, develop 브랜치로만 제한', '브랜치 보호 규칙 설정']}, {'title': '환경 설정 및 비밀 관리', 'description': '각 브랜치 및 환경에 필요한 설정을 관리하고 비밀 정보를 안전하게 저장한다.', 'business_value': '환경별 설정을 체계적으로 관리하여